# Creation of all plots in paper
This notebook do the following:
1. sync all files, stack all runs, save references
2. non-linear scale for chameleon
3. matter power spectrum


## Syncing
File are copied from `koios:/home/users/vrastil/GIT/FastSim/jobs/output/report/` and saved to `/home/michal/Documents/GIT/FastSim/report/clanek/`

In [ ]:
# import os
# import csv
# os.system("gnome-terminal -e 'bash -c \"/home/users/vrastil/GIT/FastSim/jobs/sync.sh\" '")

## Modules

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import simpy.power as pwr
import simpy.data as dt
import simpy.plot as pt
import simpy.utils as ut
from simpy import struct
import simpy.database as database

from scipy import interpolate
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

## Connect to the database

In [ ]:
db = database.connect_db()["fastsim"]

In [ ]:
# print "Num of deleted documents: ", db.data.delete_many({'type' : 'stack_info'}).deleted_count

## Stack everything
Look for new runs

In [ ]:
# local
# in_dir = '/home/michal/Documents/GIT/FastSim/jobs/output/'
# out_dir = '/home/michal/Documents/GIT/FastSim/report/clanek/'

# koios
in_dir = '/home/users/vrastil/GIT/FastSim/jobs/output/'
out_dir = '/home/users/vrastil/GIT/Paper/pix/'

In [ ]:
database.add_many_sim_data(in_dir, db)
# database.print_db_info(db)

Stack everything (according tp `query`)

In [ ]:
# all approximations, files only
query = None
skip = None
rerun = None
stack_info = dt.stack_all(db, verbose=False, query=query, skip=skip, rerun=rerun)

## Pipeline

In [ ]:
PlotOpt = pt.PlotOptions(def_dir=out_dir, show=True)
show = True

## Non-linear scale for chameleon
1. Compton wavelength $\lambda_c$
2. chameleon field $\chi_a$
3. screening potential $\Phi_{a,scr}$
4. chameleon screening scale $k_{src}$

In [ ]:
n = [0.1,0.5]
phi = [10**(-6), 10**(-5)]

dt.plot_chi_wave_pot(db, PlotOpt.append_dir('chi'), n=n, phi=phi, k_scr=False)

## Matter Power Spectrum
1. stack runs of the same approximations together (increase resolution range)
2. plot power spectrum at $z=0$ with all approximation, cut to see difference at small scales
3. plot power spectrum relative difference at different redshifts, cut to $k_{nq}/2$
4. plot power spectrum suppresion at three different scales
5. plot slope of the power spectrum

In [ ]:
PlotOpt.reset_range()
PlotOpt.set_range(ymin=0.90, ymax=1.05)
dt.get_plot_mlt_pk_broad(db, PlotOpt.append_dir('pwr_spec'), z=0, no_err=True)

In [ ]:
PlotOpt.set_range(ymin=-0.20, ymax=0.20)
dt.get_plot_pwr_spec_diff(db, PlotOpt.append_dir('pwr_spec'))

In [ ]:
si = dt.get_stack_infos(db, box_size=2000, mesh_num_pwr=1024)[0]
_, data_list = si.get_zs_data("pwr_diff", '*input*')
zs = si.data['zs']
k = data_list[0][0]
idx_up = (np.abs(k - si.k_nyquist["particle"])).argmin()
k = data_list[0][0][0:idx_up]
pt.plot_pwr_spec_diff_emu(zs, si, out_dir=out_dir + 'pwr_spec/', k=k, show=show, ymax=0.3)

## Effective redshift
1. get $a_{eff}$ for each (non-chi) stack_info
2. plot both $P(k)$--type and $\sigma_R$--type
3. plot growth rate $f$

In [ ]:
query = {'app' : dt.NON_CHI, 'type' : 'stack_info'}
database.add_smoothing_k_to_query(query)
non_chi_stack_infos = dt.get_stack_infos(db, box_size=2000, mesh_num_pwr=1024, Nt=100, query=query)
for stack_info in non_chi_stack_infos:
    dt.load_a_eff(stack_info)

pt.plot_eff_time(non_chi_stack_infos, out_dir=out_dir + 'z_eff/', a_eff_type="Pk", show=show)
# pt.plot_eff_growth_rate(non_chi_stack_infos, out_dir=out_dir + 'z_eff/', a_eff_type="Pk", show=True)

## Time-step dependence

In [ ]:
# create data object
query = {'app' : {"$in" : ['FF', 'FP']}, 'type' : 'stack_info'}
DataOpt = pt.DataOptions(data=dt.get_timestep_data(db, query=query))

# plot
pt.plot_timesteps(PlotOpt.append_dir('z_eff'), DataOpt)

## Non-linear matter power spectrum
1. stack simulations across different runs (different box sizes)
2. use effective redshift
3. plot power spectrum difference
4. plot power spectrum map
5. amplitude of non-linear power spectrum

In [ ]:
# non_chi_stack_infos = results.get_subfiles(NM=1024,  app_not='CHI', app='TZA')
# for si in non_chi_stack_infos:
#     print(si.info_tr())
#     zs, data_array = dt.transform_supp_data_to_z_eff(si)
#     dt.correct_tza(si, data_array)
#     pt.plot_pwr_spec_diff_from_data(data_array, zs, si, show_scales=False, save=False, show=True, use_z_eff=True)
#     pt.plot_pwr_spec_diff_map_from_data(data_array, zs, si, save=False, show=True, use_z_eff=True)
# dt.get_plot_mlt_pk_diff_broad(db, out_dir=out_dir + 'pwr_spec_nl/', plot_diff=False, show=show)

In [ ]:
# non_chi_stack_infos = dt.get_stack_infos(db, box_size=500, mesh_num_pwr=1024)

# for si in non_chi_stack_infos:
#     dt.get_pk_nl_amp(si)
    
# pt.plot_pwr_spec_nl_amp(non_chi_stack_infos, out_dir=out_dir + 'pwr_spec_nl/', show=True)

## Correlation function
1. simple correlation function (all approximation)
2. correlation function difference (all approximation)
3. correlation function peak amplitude and location (effective time)

In [ ]:
non_chi_stack_infos[-1].data["eff_time"]["Pk"]["D_eff_ratio"]

In [ ]:
query = {'type' : 'stack_info', 'integ_opt.time_step' : 0.01}
database.add_smoothing_k_to_query(query)
    
non_chi_stack_infos = dt.get_stack_infos(db, query=query, box_size=2000, mesh_num_pwr=1024)
dt.corr_func_comp_plot(db, {}, sim_infos=non_chi_stack_infos, outdir=out_dir + 'corr_func/', z=0, show=show, scale_to_lin=True)

In [ ]:
query = {'type' : 'stack_info', 'integ_opt.time_step' : 0.01}
database.add_smoothing_k_to_query(query)

non_chi_stack_infos = dt.get_stack_infos(db, query=query, box_size=2000, mesh_num_pwr=1024)
dt.corr_func_comp_plot_peak(db, {}, sim_infos=non_chi_stack_infos, outdir=out_dir + 'corr_func/', plot_all=False, show=show)

### Precision -- data

In [ ]:
# open CSV file for writing
filename = out_dir + 'data/precision.tex'
with open(filename, mode='w') as data_file:
    # get power spectrum precision, z = 0, k = 0.1 Mpc/h, 0.3 Mpc/h
    z = 0
    ks = [0.1, 0.2]
    non_chi_stack_infos = dt.get_stack_infos(db, box_size=2000, mesh_num_pwr=1024)
    
    # write header
    data_file.write("\\begin{tabular}{l%s}\n" % ('c' * (len(ks) + 3)))
    data_file.write("\t\\hline \\hline\n\t & ")
    
    for i in range(len(ks)):
        data_file.write("$P_%i(k)$ & " % (i + 1))
        
    data_file.write("amp & loc & width \\\\ \n\t\\hline\n")
        
    for si in non_chi_stack_infos:
        dt.init_data(si, get_pk=True)
        data_file.write("\t" + si.app)

        # get k, Pk
        zs = si.data['zs']
        z_idx = dt.find_nearest_idx(zs, z)
        data = si.data['pk_data_par'][z_idx]

        # get z_eff
        a = 1./(1.+z)
        a_idx = dt.find_nearest_idx(si.data['eff_time']['Pk']['a'], a)
        z_eff = si.data['eff_time']['Pk']['z_eff'][a_idx]
        a_eff = 1./(1. + z_eff)

        # get precision
        for k in ks:
            idx = dt.find_nearest_idx(data[0], k)
            k = data[0][idx]
            Pk = data[1][idx]
            Pk_nl = pwr.non_lin_pow_spec(a_eff, k, si.sim.cosmo)
            err = abs(Pk/Pk_nl - 1)*100
            data_file.write(' & $%.1f\\%%$' % err)

        # get BAO peak precision, z = 0, loc, amp, width
        bao = si.data['corr_func']['par_peak'][z_idx]['popt']
        bao_nl = pwr.get_bao_peak(pwr.corr_func(si.sim, z=z_eff, non_lin=True))['popt']

        # get precision
        err = np.abs(bao / bao_nl - 1)*100
        for err_ in err:
            data_file.write(' & $%.1f\\%%$' % err_)
        data_file.write(" \\\\\n")
    data_file.write("\t\\hline \\hline\n")
    data_file.write("\\end{tabular}\n")

## Chameleon results
1. effect of simulation resolution on resulting power spectrum
2. growth of structures on different scales (comparison with FPA)
3. compare different chameleon parameters (mainly through BAO peak)

### Power spectrum

In [ ]:
PlotOpt.set_range(ymin=0.7, ymax=1.7)

# non-linear CHI
query = {'app' : 'CHI', 'type' : 'stack_info'}
chi_opt = {'linear' : False}
chi_stack_infos = dt.get_stack_infos(db, query=query, chi_opt=chi_opt, mesh_num=256, box_size=2000)
chi_stack_infos = dt.sort_chi_infos(chi_stack_infos, reverse=True)
dt.plot_pwr_spec_comparison_si(chi_stack_infos, PlotOpt.append_dir('chi/fp_', out_slash=False), z=0, chi=True)

# non-linear CHI FF
query = {'app' : 'CHI_FF', 'type' : 'stack_info'}
chi_opt = {'linear' : False}
chi_stack_infos = dt.get_stack_infos(db, query=query, chi_opt=chi_opt, mesh_num=256, box_size=2000)
chi_stack_infos = dt.sort_chi_infos(chi_stack_infos, reverse=True)
dt.plot_pwr_spec_comparison_si(chi_stack_infos, PlotOpt.append_dir('chi/ff_', out_slash=False), z=0, chi=True)

### Resolution

In [ ]:
dt.compare_chi_res_FP(db, out_dir=out_dir + 'chi/', show=show, reverse=True)

In [ ]:
dt.compare_chi_res_FF(db, out_dir=out_dir + 'chi/', show=show, reverse=True)

In [ ]:
# # non-linear
# query = {'app' : 'CHI', 'type' : 'stack_info'}
# chi_opt = {'linear' : False, 'n' : 0.5, 'phi' : 1e-5}
# chi_stack_infos = dt.get_stack_infos(db, query=query, chi_opt=chi_opt)
# chi_opt['linear'] = True
# chi_stack_infos_lin = dt.get_stack_infos(db, query=query, chi_opt=chi_opt)

# fig = plt.figure(figsize=(10, 8))
# ax = plt.gca()
# ax.set_xscale('log')
# ax.set_yscale('log')
# for si, si_lin in zip(chi_stack_infos, chi_stack_infos_lin):
#     data = si.get_zs_data('pwr_spec', '*par*')[1][-1]
#     k, Pk = data[0], data[1]
#     data = si_lin.get_zs_data('pwr_spec', '*par*')[1][-1]
#     k, Pk_lin = data[0], data[1]
#     ax.plot(k, Pk/Pk_lin)
    

### Power spectrum -- vs FF, FP

In [ ]:
PlotOpt.set_range(ymin=0.98, ymax=None)

dt.compare_chi_fp(db, PlotOpt.append_dir('chi/'), mesh_num=256, z=0, show=show, reverse=True)
dt.compare_chi_fp(db, PlotOpt.append_dir('chi/'), app='FF', app_chi='CHI_FF', mesh_num=256, z=0, show=show, reverse=True)

In [ ]:
PlotOpt.set_range(ymin=0.91, ymax=1.002, xmin=2e-2)
dt.compare_chi_fp(db, PlotOpt.append_dir('chi/'), mesh_num=256, z=0, show=show, reverse=True, psl_ratio=True)
dt.compare_chi_fp(db, PlotOpt.append_dir('chi/'), app='FF', app_chi='CHI_FF', mesh_num=256, z=0, show=show, reverse=True, psl_ratio=True)

### Correlation function

In [ ]:
chi_opt = {'linear' : False, 'phi' : 1e-5, 'n' : 0.5}

query = {'app' : 'CHI', 'type' : 'stack_info'}
chi_stack_infos = dt.get_stack_infos(db, query=query, chi_opt=chi_opt, mesh_num=256, box_size=2000)
dt.corr_func_comp_plot(db, {}, sim_infos=chi_stack_infos, outdir=out_dir + 'chi/', z=0, show=show, scale_to_lin=True)

query = {'app' : 'CHI_FF', 'type' : 'stack_info'}
chi_stack_infos = dt.get_stack_infos(db, query=query, chi_opt=chi_opt, mesh_num=256, box_size=2000)
dt.corr_func_comp_plot(db, {}, sim_infos=chi_stack_infos, outdir=out_dir + 'chi/', z=0, show=show, scale_to_lin=True)

In [ ]:
####################
# CHI vs FP
###################
query = {'app' : 'CHI', 'type' : 'stack_info'}

# ylim
yrange = {
    'amp' : (-0.16, 1.23),
    'loc' : (-0.008, 0.053),
    'width' : (-0.72, 0.12)
}

# non-linear
chi_opt = {'linear' : False}
chi_stack_infos = dt.get_stack_infos(db, query=query, chi_opt=chi_opt, mesh_num=256, box_size=2000)
dt.corr_func_comp_plot_peak(db, None, sim_infos=chi_stack_infos, outdir=out_dir + 'chi/nl_fp_', plot_all=False, chi=True, yrange=yrange, show=show, reverse=True)

# # linear
# chi_opt = {'linear' : True}
# chi_stack_infos = dt.get_stack_infos(db, query=query, chi_opt=chi_opt, mesh_num=256)
# dt.corr_func_comp_plot_peak(db, None, sim_infos=chi_stack_infos, outdir=out_dir + 'chi/lin_fp_', plot_all=False, chi=True, yrange=yrange, show=show, reverse=True)

####################
# CHI_FF vs FF
###################
query = {'app' : 'CHI_FF', 'type' : 'stack_info'}

# non-linear
chi_opt = {'linear' : False}
chi_stack_infos = dt.get_stack_infos(db, query=query, chi_opt=chi_opt, mesh_num=256, box_size=2000)
dt.corr_func_comp_plot_peak(db, None, sim_infos=chi_stack_infos, outdir=out_dir + 'chi/nl_ff_', plot_all=False, chi=True, yrange=yrange, show=show, reverse=True)

# linear
# chi_opt = {'linear' : True}
# chi_stack_infos = dt.get_stack_infos(db, query=query, chi_opt=chi_opt, mesh_num=256)
# dt.corr_func_comp_plot_peak(db, None, sim_infos=chi_stack_infos, outdir=out_dir + 'chi/lin_ff_', plot_all=False, chi=True, yrange=yrange, show=show, reverse=True)

### Correlation function -- vs FPA

In [ ]:
# dt.corr_func_chi_fp_plot_peak(db, out_dir=out_dir + "chi/lin", mesh_num=256, chi_opt={'linear' : True})
# dt.corr_func_chi_fp_plot_peak(db, out_dir=out_dir + "chi/nl", mesh_num=256, chi_opt={'linear' : False}, yrange=yrange, show=show, reverse=True)

### Power spectrum -- map vs FP

In [ ]:
chi_info = dt.get_stack_infos(db, 
            query={'app' : 'CHI', 'type' : 'stack_info'},
            chi_opt={'linear' : False, 'n' : 0.5, 'phi' : 1e-5},
            mesh_num=512, box_size=500
)[0]

fp_info = dt.get_stack_infos(db, 
            query={'app' : 'FP', 'type' : 'stack_info'},
            mesh_num=512, box_size=500
)[0]

dt.compare_chi_fp_map(chi_info, fp_info, out_dir=out_dir + "chi/", show=show)

In [ ]:
# chi_info = dt.get_stack_infos(db, 
#             query={'app' : 'CHI_FF', 'type' : 'stack_info'},
#             chi_opt={'linear' : False, 'n' : 0.5, 'phi' : 1e-5},
#             mesh_num=256, box_size=2000
# )[0]

# ff_info = dt.get_stack_infos(db, 
#             query={'app' : 'FF', 'type' : 'stack_info'},
#             mesh_num=256, box_size=2000
# )[0]

# dt.compare_chi_fp_map(chi_info, ff_info, out_dir=out_dir + "chi/", show=show)

## Sigma8

In [ ]:
# # param
# mesh_num = 512
# box_size = 2000
# z = 0

# apps = ["ZA", "TZA", "FP"]
# sim_infos = []

# # get SimInfo
# for app in apps:
#     si = dt.get_stack_infos(db, 
#             query={'app' : app, 'type' : 'stack_info'},
#             mesh_num=mesh_num, box_size=box_size
#     )[0]
#     # initialize data
#     dt.init_data(si, get_sigma=True)

#     sim_infos.append(si)

In [ ]:
# for si in sim_infos:
#     app = si.app
#     # sigma 8
#     idx_z = dt.find_nearest_idx(si.data["sigma_R"]["zs"], z)
#     idx_8 = dt.find_nearest_idx(si.data["sigma_R"]["par"][idx_z][0], 8)
#     R8 = si.data["sigma_R"]["par"][idx_z][0][idx_8]
#     sigma_8 = si.data["sigma_R"]["par"][idx_z][1][idx_8]
    
#     # Pk amplitude
#     ExtraPk = si.data["pk_list"][-1]
#     A_low = ExtraPk.A_low

#     # display
#     print("%s: R = %.2f Mpc/h, sigma = %.6e, A = %.3f" % (app, R8, sigma_8, A_low))


In [ ]:
# sim.cosmo.sigma8

In [ ]:
# fig = plt.figure(figsize=(10, 8))
# ax = plt.gca()
# R8 = 8
# ax.set_xscale('log')
# # ax.set_yscale('log')
# zs = np.logspace(-1, 2)
# sigma_ratios = []
# for z in zs:
#     data = pwr.sigma_R(sim, z=z)
#     fce = interpolate.interp1d(data[0], data[1])
#     D = pwr.growth_factor(1./(1.+z), sim.cosmo)
#     sigma_a = sim.cosmo.sigma8*D*D
#     sigma_8 = fce(R8)
#     sigma_ratios.append(sigma_8/sigma_a)
    
# ax.plot(zs, sigma_ratios)

## Snapshots

In [ ]:
# get seed
# seed = db.data_large.find_one({}, {'_id' : 0, 'run_opt.seed' : 1})['run_opt']['seed']
# query = {'run_opt.seed' : seed}
# print "Num of deleted documents (data_large): ", db.data_large.delete_many({}).deleted_count
# print "Num of deleted documents (data): ", db.data.delete_many(query).deleted_count

In [ ]:
# print "Num of deleted documents: ", db.data_large.delete_many({}).deleted_count
database.print_db_info(db, collection='data_large')

In [ ]:
# parameters
# query = {'app' : 'ZA', 'box_opt.box_size' : 2000}
query = {'app' : 'FP', 'box_opt.box_size' : 200}
# query = {}
collection = 'data_large'
Nt = 200

In [ ]:
dt.analyze_all(db, query=query, collection=collection, rerun='dens_slice', skip='dens_ani', show=True, save=True,
               out_dir=out_dir+'dens/')

## Non-linear scale

In [ ]:
from scipy.integrate import quad
from scipy.optimize import brentq

def get_power_spectrum_fce():
    query = {'app' : {'$nin' : ['TZA', 'CHI', 'CHI_FF']}, 'type' : 'stack_info'}
    doc = db['data'].find_one(query, {'_id' : 1})
    si = struct.StackInfo(db, doc)
    cosmo = si.sim.cosmo
    Pk = lambda k: pwr.lin_pow_spec(1, k, cosmo)
    return Pk

def get_power_spectrum_integral():
    Pk = get_power_spectrum_fce()
    func = lambda k: k*k*Pk(k)/(2*np.pi**2)
    Pk_int = lambda k: quad(func, 0, k)
    return Pk_int

def find_k_non_linear():
    Pk_int = get_power_spectrum_integral()
    func = lambda k: Pk_int(k)[0] - 1
    k_nl = brentq(func, 1e-3, 1)
    return k_nl

In [ ]:
find_k_non_linear()

In [ ]:
find_k_non_linear()*1.25

In [ ]:
find_k_non_linear()*1.5

In [ ]:
"http://adlibitum.oats.inaf.it/monaco/Papers/monaco.2016.Galaxies.4.53.pdf"